In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import torchinfo
import my

In [2]:
df = pd.read_csv('./ai_job_market.csv')
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,company_size,tools_preferred
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,Large,"KDB+, LangChain"
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,Large,"FastAPI, KDB+, TensorFlow"
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,Large,"BigQuery, PyTorch, Scikit-learn"
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,Large,"TensorFlow, BigQuery, MLflow"
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,Large,"PyTorch, LangChain"
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,Large,MLflow
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,Large,"KDB+, LangChain, MLflow"
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,Large,PyTorch
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,Large,"MLflow, TensorFlow, FastAPI"


In [3]:

df['salary_range_usd']
df[['Min_Salary', 'Max_Salary']] = df['salary_range_usd'].str.split('-', expand=True)
df['Min_Salary'] = pd.to_numeric(df['Min_Salary'], errors='coerce')
df['Max_Salary'] = pd.to_numeric(df['Max_Salary'], errors='coerce')
df['Average_Salary']=((df["Min_Salary"]+df["Max_Salary"])/20000)


skills=df["skills_required"].str.split(',', expand=True)
skills.columns = [f'Skill_{i+1}' for i in range(skills.shape[1])]
tools=df["tools_preferred"].str.split(',', expand=True)
tools.columns = [f'Tools_{i+1}' for i in range(tools.shape[1])]
skills
df['Average_Salary']


0       10.12290
1       11.16990
2       17.08500
3        8.73255
4       11.70535
          ...   
1995    10.02540
1996     9.25215
1997    15.75510
1998     7.24635
1999     7.66045
Name: Average_Salary, Length: 2000, dtype: float64

In [4]:

df = pd.concat([df, skills,tools], axis=1)
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,...,Average_Salary,Skill_1,Skill_2,Skill_3,Skill_4,Skill_5,Skill_6,Tools_1,Tools_2,Tools_3
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,...,10.12290,NumPy,Reinforcement Learning,PyTorch,Scikit-learn,GCP,FastAPI,KDB+,LangChain,None
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,...,11.16990,Scikit-learn,CUDA,SQL,Pandas,None,None,FastAPI,KDB+,TensorFlow
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,...,17.08500,MLflow,FastAPI,Azure,PyTorch,SQL,GCP,BigQuery,PyTorch,Scikit-learn
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,...,8.73255,Scikit-learn,C++,Pandas,LangChain,AWS,R,TensorFlow,BigQuery,MLflow
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,...,11.70535,Excel,Keras,SQL,Hugging Face,None,None,PyTorch,LangChain,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,...,10.02540,Flask,FastAPI,Power BI,None,None,None,MLflow,None,None
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,...,9.25215,R,Flask,Excel,C++,CUDA,Scikit-learn,KDB+,LangChain,MLflow
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,...,15.75510,Hugging Face,Excel,Scikit-learn,R,MLflow,None,PyTorch,None,None
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,...,7.24635,AWS,Python,Scikit-learn,None,None,None,MLflow,TensorFlow,FastAPI


In [5]:
label1, uniqueValues1= pd.factorize(df['Skill_1'])
label2, uniqueValues2= pd.factorize(df['Skill_2'])
label3, uniqueValues3= pd.factorize(df['Skill_3'])
label4, uniqueValues4= pd.factorize(df['Skill_4'])
label5, uniqueValues5= pd.factorize(df['Skill_5'])
label6, uniqueValues6= pd.factorize(df['Tools_1'])
label7, uniqueValues7= pd.factorize(df['Tools_2'])
label8, uniqueValues8= pd.factorize(df['Tools_3'])
label9, uniqueValues9= pd.factorize(df['location'])
label10, uniqueValues10= pd.factorize(df['employment_type'])
label11, uniqueValues11= pd.factorize(df['job_title'])
label12, uniqueValues12= pd.factorize(df['industry'])
label13, uniqueValues13= pd.factorize(df['company_name'])



In [6]:
targetlabel, targetunique= pd.factorize(df['Average_Salary'])
df['target']=targetlabel
df['target']
targetunique

Index([ 10.1229,  11.1699,   17.085,  8.73255, 11.70535,  13.6675,  7.95845,
         7.2588, 16.13715,  12.2195,
       ...
        16.5924,  11.4291,  5.49335,  19.1729,  10.7884,  10.0254,  9.25215,
        15.7551,  7.24635,  7.66045],
      dtype='float64', length=1992)

In [7]:
df['fskill_1']=label1
df['fskill_2']=label2
df['fskill_3']=label3
df['fskill_4']=label4
df['fskill_5']=label5
df['ftool_1']=label6
df['ftool_2']=label7
df['ftool_3']=label8
df['floc']=label9
df['ftype']=label10
df['ftitle']=label11
df['findustry']=label12
df['fname']=label13


In [8]:
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,...,fskill_4,fskill_5,ftool_1,ftool_2,ftool_3,floc,ftype,ftitle,findustry,fname
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,...,0,0,0,0,-1,0,0,0,0,0
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,...,1,-1,1,1,0,1,0,1,1,1
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,...,2,1,2,2,1,2,0,2,1,2
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,...,3,2,3,3,2,3,0,3,1,3
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,...,4,-1,4,0,-1,4,1,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,...,-1,-1,6,-1,-1,1995,3,6,2,1904
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,...,13,13,0,0,2,1996,2,3,4,1905
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,...,19,7,4,-1,-1,1997,1,0,5,1906
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,...,-1,-1,6,4,3,1998,1,2,5,1907


In [9]:
#df= df.drop(columns=['floc'], axis=1)
inputs = torch.tensor(df.iloc[:,-12:].values, dtype=torch.float32)
df.iloc[:,-12:]



,fskill_2,fskill_3,fskill_4,fskill_5,ftool_1,ftool_2,ftool_3,floc,ftype,ftitle,findustry,fname
0,0,0,0,0,0,0,-1,0,0,0,0,0
1,1,1,1,-1,1,1,0,1,0,1,1,1
2,2,2,2,1,2,2,1,2,0,2,1,2
3,3,3,3,2,3,3,2,3,0,3,1,3
4,4,1,4,-1,4,0,-1,4,1,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2,14,-1,-1,6,-1,-1,1995,3,6,2,1904
1996,6,20,13,13,0,0,2,1996,2,3,4,1905
1997,5,4,19,7,4,-1,-1,1997,1,0,5,1906
1998,16,4,-1,-1,6,4,3,1998,1,2,5,1907


In [10]:

targets =  torch.tensor(df['Average_Salary'].values, dtype=torch.int64)

targets
max_target_value = max(targets) 
print(f"Maximum target value found: {max_target_value}")

Maximum target value found: 19


In [11]:
dataset =  TensorDataset(inputs, targets)

In [12]:
dataset[0]
for i in range (1999,2000):
    print(dataset[i])

(tensor([ 2.0000e+01,  1.4000e+01, -1.0000e+00, -1.0000e+00,  6.0000e+00,
        -1.0000e+00, -1.0000e+00,  1.9990e+03,  0.0000e+00,  2.0000e+00,
         2.0000e+00,  1.9080e+03]), tensor(7))


In [13]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [14]:
torch.manual_seed(0)
next(iter(dataloader))

[tensor([[ 1.1000e+01,  6.0000e+00, -1.0000e+00, -1.0000e+00,  7.0000e+00,
           3.0000e+00,  2.0000e+00,  1.2260e+03,  2.0000e+00,  2.0000e+00,
           5.0000e+00,  1.1920e+03],
         [ 1.6000e+01,  7.0000e+00,  3.0000e+00, -1.0000e+00,  3.0000e+00,
           2.0000e+00,  5.0000e+00,  5.0900e+02,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  5.0300e+02],
         [ 2.1000e+01,  1.5000e+01,  0.0000e+00, -1.0000e+00,  6.0000e+00,
           4.0000e+00,  5.0000e+00,  1.6310e+03,  0.0000e+00,  1.0000e+00,
           6.0000e+00,  1.5690e+03],
         [ 6.0000e+00,  1.9000e+01, -1.0000e+00, -1.0000e+00,  4.0000e+00,
           5.0000e+00,  0.0000e+00,  1.1320e+03,  1.0000e+00,  6.0000e+00,
           5.0000e+00,  1.1040e+03]]),
 tensor([11,  7, 13,  5])]

In [15]:
class LinearClassifier(nn.Module):
    def __init__(self):
        super(LinearClassifier, self).__init__()
        # Ensure 'num_classes' is set correctly (e.g., to 20 if targets go up to 19)
        self.fc = nn.Linear(12, 20) 

    def forward(self, x):
        # Flatten the input if necessary (e.g. for image data going into a Linear layer)
        x = x.view(x.size(0), -1) 
        x = self.fc(x)
        return x

In [16]:
torchinfo.summary(LinearClassifier())

Layer (type:depth-idx)                   Param #
LinearClassifier                         --
├─Linear: 1-1                            260
Total params: 260
Trainable params: 260
Non-trainable params: 0

In [17]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = LinearClassifier()
y = model(x)
y


tensor([[ -59.3076,  -49.4266, -168.3617,  547.9552,  -78.4537,  178.9194,
         -512.3290, -427.2602,  -67.1335,  245.5421,   35.3349,  -25.0336,
         -483.5486,   -7.2059, -185.6961,   52.6743,   45.7881, -232.0408,
          182.8819,  107.0293],
        [ -25.1984,  -21.9526,  -72.8711,  229.8747,  -30.4634,   76.4509,
         -212.9668, -178.7315,  -23.4610,  103.3546,    9.5224,   -9.1716,
         -204.3614,   -6.3446,  -73.6089,   23.9427,   18.2451,  -96.9822,
           78.6875,   46.1385],
        [ -78.2141,  -60.1128, -228.0318,  724.6220, -105.7067,  232.5852,
         -679.3525, -567.6210,  -88.9095,  325.3935,   45.0346,  -32.6958,
         -643.6375,   -8.4325, -245.7051,   76.2679,   55.9054, -303.3461,
          241.9416,  143.2538],
        [ -53.4373,  -44.0329, -153.4414,  504.8410,  -72.0515,  166.4881,
         -476.5501, -397.5427,  -58.6667,  231.4824,   31.3877,  -24.5602,
         -450.1397,   -6.9376, -171.6958,   49.5467,   46.8610, -215.5497,
    

In [18]:
import torch.nn.functional as F
def loss_fn(y, targets): # 'y' is the first parameter name
    loss = F.cross_entropy(y, targets)
    return loss

In [19]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = LinearClassifier()

# 1. Get the model's predictions
y = model(x) 

with torch.no_grad():
    # 2. Pass 'y' (the predictions/logits) into the loss function, not 'x' (the input data)
    loss = loss_fn(y, targets) 
    
print(loss)

tensor(513.2506)


In [20]:
def accuracy(y, targets):
# 1. Convert logits to predicted class indices
    predictions = torch.argmax(y, dim=1)
    
    # 2. Compare predictions with actual targets (boolean tensor)
    correct_predictions = (predictions == targets)
    
    # 3. Count correct predictions using .sum(). 
    # This result is already a single-element LongTensor.
    num_correct_tensor = correct_predictions.sum()
    
    # 4. Calculate accuracy using tensor division.
    # Convert num_correct_tensor to float for the division operation
    acc_tensor = num_correct_tensor.float() / targets.size(0)
    
    return acc_tensor

In [21]:
# [YOUR WORK HERE]
# @workUnit

def train(model, dataloader, epochs:int, lr:float, debug=False):
        # Set the model to training mode (important for layers like BatchNorm, Dropout)
    model.train()

    # Define the Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

  

    for epoch in range(epochs):
        
        running_loss = 0.0
        running_corrects = 0
        total_samples_epoch = 0

        # Iterate over the data in batches
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            
            # --- Training steps for a single batch ---

            # 1. Zero the parameter gradients
            optimizer.zero_grad()

            # 2. Forward pass: compute the model output (logits)
            y_logits = model(inputs)

            # 3. Calculate loss
            loss = loss_fn(y_logits, targets)
            
            # 4. Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()

            # 5. Optimizer step: update model parameters
            optimizer.step()


In [22]:
# [THIS IS READ-ONLY]
# @check

torch.manual_seed(0)
model = LinearClassifier()
x_all, targets_all = dataset[:]
y_all = model(x_all)
print("Before training, accuracy =", accuracy(y_all, targets_all))

train(model, dataloader, epochs=4, lr=0.01, debug=False)

y_all = model(x_all)
print("After training, accuracy =", accuracy(y_all, targets_all))

Before training, accuracy = tensor(0.0165)
After training, accuracy = tensor(0.0845)
